## *PhotoDissociation Region Toolbox Notebooks*
---------------------------------------------------------------

# Example 2: Introduction to ModelSets

PDRT supports a variety of PDR models to be used to fit your data.  

The Wolfire/Kaufman 2020 (WK2020) ModelSet contains constant density models for metallicity $z=1$. There are also a WK2006 z=3 and z=0.1 sets, with limited spectral lines. Models are stored as ratios of intensities as a function of radiation field $I_{FUV}$ and hydrogen nucleus volume density $n$. In some cases, the individual intensity models are also available (but fitting with `LineRatioFit` only uses the ratios). We will continue to expand 2020 model set to include a wider range of lines and metallicities.  

We also have available [KOSMA-tau](https://astro.uni-koeln.de/stutzki/research/kosma-tau)  ModelSets from Markus Rollig. These are both 'clumpy' and non-clumpy' models with varying mass and $R_v$ value and $z=1$. In a non-clumpy model the mass is of the spherical clump. The density (profile) and the mass determine the clump radius and the total $A_V$ to the clump center. In the clumpy model the mass is the total (ensemble) mass of all clumps which are distributed according to a power law with the mass range $[M_{low}, M_{up}]$. 

Any PDR models
can be used if they are stored in the [correct FITS format](https://docs.google.com/document/d/1hPSUip5RK6nPmqrL8iw3qkX69DVLBv0sAuqac18pLT8/edit?usp=sharing). 
 

In [ ]:
from pdrtpy.modelset import ModelSet
from pdrtpy import pdrutils as utils

### list() describes all the ModelSets available. 

In [ ]:
ModelSet.all_sets().show_in_notebook()

### Instantiate the KOSMA-tau clumpy, 10 $M_\odot$, $R_v$=5.5 models for z=1. 

In [ ]:
ms = ModelSet("kt2013wd01-25",medium="clumpy",mass=10,z=1)
#ms = ModelSet("wk2020",z=1)

### Show the model intensities and ratios that are available in this ModelSet
`supported_intensities` and `supported_ratios` is an astropy Table, invoke *show_in_notebook()* to see all the rows.  There may be more ratios than lines because not all components of a ratio may be available as separate models.

In [ ]:
ms.supported_intensities

In [ ]:
ms.supported_ratios.show_in_notebook()

Every model has a one-line description. You can query a ModelSet to find which spectral lines (your observations) are supported and which model ratios it has for your lines.

In [ ]:
print(ms.description)

mylines =["CII_158","OI_145","FIR","CI_609","CO_65"]
print("There are %d ratios in the %s ModelSet for the lines %s:"%(ms.ratiocount(mylines),ms.name,mylines))
print(ms.model_ratios(mylines))

# If you want to see the actual FITS file names. The root of the path is where your pdrtpy is installed.
print("\nThey are located in:")
for p in ms.find_files(mylines):
    print(p)

## How to access a single model
This example returns the model for \[C II\] 158 $\mu$m/CO(4-3) intensity ratio. The access key is the tag in the `ratio label` in the supported ratios table above.  The model is returned as a `pdrtpy.measurement.Measurement`

In [ ]:
cico = ms.get_model("CII_158/CO_43")
print(f"{cico.id} Min value:{cico.data.min():.1e} Max value:{cico.data.max():.1e}")
print(f"Density units: {cico.header['CUNIT1']}, radiation field units: {cico.header['CUNIT2']}")

## Probe the ratio or intensity values in a model
Once you have a model you can use the `get` function to discover the model value at a particular density and radiation field.  You must give the density and radiation field in the units of the model.  For WK2020, this is cm$^{-3}$ and erg cm$^{-2}$ s$^{-1}$.  However, you can use astropy's `Quantity` to convert for you.

In [ ]:
n=1234.5
rf=0.6789
cico.get(n,rf)

#### Now suppose you want to use Habing for the radiation field
Use Quantity to convert and pass the value to `get`.

In [ ]:
from astropy.units import Quantity
rf=Quantity(2.3,'Habing')
print(f"{rf.to(cico.header['CUNIT2']):0.2e}")
cico.get(n,rf.to(cico.header['CUNIT2']).value)

### How to access multiple models
Suppose you want to find all the models that use some combination of CO(J=1-0), \[C II\] 158 $\mu$m, \[O I\] 145 $\mu$m, and $I_{FIR}$. This example gets both intensity and ratio models, though you can specify `model_type='intensity'` or `model_type='ratio'` to get one or the other.  The models are returns as a dictionary with the keys set to the model IDs.

In [ ]:
mods = ms.get_models(["CII_158","OI_145", "CO_10", "FIR"],model_type='both')

#### The returned models are three ratio models and one intensity model

In [ ]:
mods.keys()

In [ ]:
# average value of one of the models
print(mods['CII_158/CO_10'].data.mean())